In [6]:
import torch
import torchvision.models as models


In [3]:
model = models.densenet121(pretrained=False)

/system/linux/anaconda3.7/lib/python3.7/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [ ]:
weights = 'bestfitting/models/external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/fold0/final.pth'
model.load_state_dict(torch.load(weights)['state_dict'])

In [15]:
d = torch.load(weights)

In [27]:
# mabe we just have to remove the word backbone?
d['state_dict'].keys()
d2 = {}
for key in d['state_dict']:
    if 'backbone' in key:
        d2[key[9:]] = d['state_dict'][key]
    else:
        d2[key] = d['state_dict'][key]


In [ ]:
d2.keys()